In [366]:
import pandas as pd

In [467]:
train = pd.read_csv("data/train.csv", parse_dates=["datetime"])

# parse_dates=["datetime"]을 사용하여 파싱해서 데이터를 사용할 수 있다.
# train["datetime"].dt.year
# train["datetime"].dt.month
# train["datetime"].dt.day

print(train.shape)
train.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [468]:
test = pd.read_csv("data/test.csv", parse_dates=["datetime"])

# shape를 찍으면 가로세로 로우 수가 나오기 때문에 어떤 데이터를 로드하고 있는지 알 수 있음
# 그래서 shape를 찍는 것을 권장한다.
# train데이터와 test데이터의 차이 우리가 맞추는 것은 자전거 대여량을 예측해야함
# 캐주얼/레지스터는 비회원/회원의 대여량임 
# 카운트에 대한 데이터는 사용하기 어려운데 어떻게 사용할지 고민을 해보는 게 좋음

print(test.shape)
test.head()

(6493, 9)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [469]:
print(train["temp"].min())
print(train["temp"].mean())
print(train["temp"].max())
print("-----")
print(train["humidity"].min())
print(train["humidity"].mean())
print(train["humidity"].max())
print("-----")
print(train["windspeed"].min())
print(train["windspeed"].mean())
print(train["windspeed"].max())
print("-----")
print(train["weather"].min())
print(train["weather"].mean())
print(train["weather"].max())


train["temp_Bad"] = (train["temp"] < 10) | (train["temp"] >= 32)
train["temp_Good"] = (train["temp"] >= 10) & (train["temp"] < 32 )

train["humidity_Bad"] = (train["humidity"] < 30) | (train["humidity"] > 80)
train["humidity_Good"] = (train["humidity"] >= 30) & (train["humidity"] <= 80 )

train["windspeed_Low"] = train["windspeed"] < 30
train["windspeed_High"] = train["windspeed"] >= 30

0.82
20.23085982
41.0
-----
0
61.886459673
100
-----
0.0
12.7993954069
56.9969
-----
1
1.41842733787
4


In [470]:
test["temp_Bad"] = (test["temp"] < 10) | (test["temp"] >= 32)
test["temp_Good"] = (test["temp"] >= 10) & (test["temp"] < 32 )


test["humidity_Bad"] = (test["humidity"] < 30) | (test["humidity"] > 80)
test["humidity_Good"] = (test["humidity"] >= 30) & (test["humidity"] <= 80 )


test["windspeed_Low"] = test["windspeed"] < 30
test["windspeed_High"] = test["windspeed"] >= 30

In [471]:
# train 월 인코딩
train["year"] = train["datetime"].dt.year
train["month"] = train["datetime"].dt.month
train["hour"] = (train["datetime"].dt.hour > 21) | (train["datetime"].dt.hour < 6) == 0



train["season1"] = train["season"] == 1
train["season2"] = train["season"] == 2
train["season3"] = train["season"] == 3
train["season4"] = train["season"] == 4


train["weather1"] = train["weather"] == 1
train["weather2"] = train["weather"] == 2
train["weather3"] = train["weather"] == 3
train["weather4"] = train["weather"] == 4

train["weekday"] = train["datetime"].dt.weekday
train["weekday1"] = train["datetime"].dt.weekday == 1
train["weekday2"] = train["datetime"].dt.weekday == 2
train["weekday3"] = train["datetime"].dt.weekday == 3
train["weekday4"] = train["datetime"].dt.weekday == 4
train["weekday5"] = train["datetime"].dt.weekday == 5
train["weekday6"] = train["datetime"].dt.weekday == 6
train["weekday7"] = train["datetime"].dt.weekday == 7
train["hour"] = train["datetime"].dt.hour
print(train.shape)

train[["year", "month", "weekday1","weekday2","weekday3","weekday4","weekday5","weekday7","weekday7", "hour"]].tail()


(10886, 37)


,year,month,weekday1,weekday2,weekday3,weekday4,weekday5,weekday7,weekday7,hour
10881,2012,12,False,True,False,False,False,False,False,19
10882,2012,12,False,True,False,False,False,False,False,20
10883,2012,12,False,True,False,False,False,False,False,21
10884,2012,12,False,True,False,False,False,False,False,22
10885,2012,12,False,True,False,False,False,False,False,23


In [472]:
#  test 월 인코딩
test["year"] = test["datetime"].dt.year
test["month"] = test["datetime"].dt.month


test["hour"] = (test["datetime"].dt.hour > 21) | (test["datetime"].dt.hour < 6) == 0


test["weather1"] = test["weather"] == 1
test["weather2"] = test["weather"] == 2
test["weather3"] = test["weather"] == 3
test["weather4"] = test["weather"] == 4

test["season1"] = test["season"] == 1
test["season2"] = test["season"] == 2
test["season3"] = test["season"] == 3
test["season4"] = test["season"] == 4

test["weekday"] = test["datetime"].dt.weekday
test["weekday1"] = test["datetime"].dt.weekday == 1
test["weekday2"] = test["datetime"].dt.weekday == 2
test["weekday3"] = test["datetime"].dt.weekday == 3
test["weekday4"] = test["datetime"].dt.weekday == 4
test["weekday5"] = test["datetime"].dt.weekday == 5
test["weekday6"] = test["datetime"].dt.weekday == 6
test["weekday7"] = test["datetime"].dt.weekday == 7
test["hour"] = test["datetime"].dt.hour
print(test.shape)

test[["year", "month", "weekday1","weekday2","weekday3","weekday4","weekday5","weekday6","weekday7", "hour"]].head()

(6493, 34)


,year,month,weekday1,weekday2,weekday3,weekday4,weekday5,weekday7,weekday7,hour
0,2011,1,False,False,True,False,False,False,False,0
1,2011,1,False,False,True,False,False,False,False,1
2,2011,1,False,False,True,False,False,False,False,2
3,2011,1,False,False,True,False,False,False,False,3
4,2011,1,False,False,True,False,False,False,False,4


# train

In [500]:
# feature_names = ["year", "month", "weekday", "hour", "season", "holiday", "workingday", "weather", "temp", "atemp", "humidity", "windspeed"]
# # feature_names = ["year", "weekday1","weekday2","weekday3","weekday4","weekday5","weekday7","weekday7", 
#                  "season1", "season2", "season3", "season4", "holiday", "workingday",
#                  "weather1", "weather2", "weather3", "weather4", 
#                  "temp_Good", "temp_Bad", "humidity_Bad", "humidity_Good", "windspeed_Low", "windspeed_High"]
feature_names = ["year", "month", "hour", "season1", "season2", "season3", "season4", "holiday", "workingday",
                 "weather1", "weather2", "weather3", "weather4", "weekday1","weekday2","weekday3","weekday4","weekday5","weekday7","weekday7", 
                 "temp_Good", "temp_Bad", "humidity_Bad", "humidity_Good", "windspeed_Low", "windspeed_High"]
feature_names

['year',
 'month',
 'hour',
 'season1',
 'season2',
 'season3',
 'season4',
 'holiday',
 'workingday',
 'weather1',
 'weather2',
 'weather3',
 'weather4',
 'weekday1',
 'weekday2',
 'weekday3',
 'weekday4',
 'weekday5',
 'weekday7',
 'weekday7',
 'temp_Good',
 'temp_Bad',
 'humidity_Bad',
 'humidity_Good',
 'windspeed_Low',
 'windspeed_High']

In [501]:
# X_train
X_train = train[feature_names]

print(X_train.shape)
X_train.head()

(10886, 26)


,year,month,hour,season1,season2,season3,season4,holiday,workingday,weather1,...,weekday4,weekday5,weekday7,weekday7,temp_Good,temp_Bad,humidity_Bad,humidity_Good,windspeed_Low,windspeed_High
0,2011,1,0,True,False,False,False,0,0,True,...,False,True,False,False,False,True,True,False,True,False
1,2011,1,1,True,False,False,False,0,0,True,...,False,True,False,False,False,True,False,True,True,False
2,2011,1,2,True,False,False,False,0,0,True,...,False,True,False,False,False,True,False,True,True,False
3,2011,1,3,True,False,False,False,0,0,True,...,False,True,False,False,False,True,False,True,True,False
4,2011,1,4,True,False,False,False,0,0,True,...,False,True,False,False,False,True,False,True,True,False


In [495]:
# X_test
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

(6493, 25)


,year,hour,season1,season2,season3,season4,holiday,workingday,weather1,weather2,...,weekday4,weekday5,weekday7,weekday7,temp_Good,temp_Bad,humidity_Bad,humidity_Good,windspeed_Low,windspeed_High
0,2011,0,True,False,False,False,0,1,True,False,...,False,False,False,False,True,False,False,True,True,False
1,2011,1,True,False,False,False,0,1,True,False,...,False,False,False,False,True,False,False,True,True,False
2,2011,2,True,False,False,False,0,1,True,False,...,False,False,False,False,True,False,False,True,True,False
3,2011,3,True,False,False,False,0,1,True,False,...,False,False,False,False,True,False,False,True,True,False
4,2011,4,True,False,False,False,0,1,True,False,...,False,False,False,False,True,False,False,True,True,False


In [496]:
# 벡터는 소문자를 사용한다.
label_name = "count"

y_train = train[label_name] 

print(y_train.shape)
y_train.head()

(10886,)


0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

In [497]:
# DecisionTreeRegressor를 사용하는 문제
# 샘플링 할때 랜덤으로 샘플링하기 때문에 결과가 다르게 나타난다.
# 실습해서 개선시켜야 하는데 랜덤이 잘 나온건지 컬럼을 변경해서 잘 나온건지 구분해야 하기 때문에 랜덤을 고정시켜주어야 한다.
# RandomForestRegressor(random_state=38)
# 같은 시드 넘버를 고정해서 실행하면 어느 PC에서 실행하든 같은 결과가 나온다.
# 캐글 경진대회 1등이 lucky number를 포스트모템에서 공유하기도 한다.

# n_jobs 코어에 대한 옵션을 주어 사용가능한 모든 코어를 사용하게 하는 것 
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# model = DecisionTreeRegressor()
model = RandomForestRegressor(random_state=37, n_jobs=-1)

model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=37, verbose=0, warm_start=False)

In [498]:
# Score

In [499]:
from sklearn.cross_validation import cross_val_score

score = cross_val_score(model, X_train, y_train, cv=20, scoring="neg_mean_absolute_error").mean()

# 0에 근접할수록 좋은 데이터
# 마이너스 데이터가 나오는 이유는 파이콘 강연을 참고할 것
print("Score= {0:.5f}".format(score))

Score= -42.93092


In [480]:
# 학습시킴, 피팅(옷을 맞출 때 사용하는 피팅을 생각함) - 피처와 레이블을 넣어주면 알아서 학습을 함
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=37, verbose=0, warm_start=False)

In [481]:
# 예측
# shape를 찍어서 row 수를 보고 train 데이터인지 test데이터인지를 판별
predictions = model.predict(X_test)

print(predictions.shape)
predictions[0:10]

(6493,)


array([  17.2       ,    5.        ,    7.9       ,    3.4       ,
          3.6       ,    3.1       ,   31.        ,   74.1       ,
        194.23333333,  126.2       ])

In [482]:
# Shubmit

In [483]:
submission = pd.read_csv("data/sampleSubmission.csv")
submission

submission["count"] = predictions

print(submission.shape)
submission.head()

(6493, 2)


,datetime,count
0,2011-01-20 00:00:00,17.2
1,2011-01-20 01:00:00,5.0
2,2011-01-20 02:00:00,7.9
3,2011-01-20 03:00:00,3.4
4,2011-01-20 04:00:00,3.6


In [485]:
# 타이타닉 문제와 다르게 인덱스로 잡을 만한 컬럼이 없음
# 그런데 맨 앞에 index가 있으면 캐글에서 오류가 나기 때문에 index를 없애고 데이터를 추출한다.
submission.to_csv("data/baseline_script_rf2.csv", index=False)

측정방식은 classification과 Regression에 따라 다름
Regression은 3개 정도 알면 거기서 발전시켜 나감

actual = 100 
